In [4]:
import subprocess
import os
import glob
import sys
import time
import pandas as pd
import numpy as np
import datetime as dt
from sortedcontainers import SortedSet
import gc

from decord import VideoReader, cpu, gpu
import cv2
import subprocess as sp
from tqdm import tqdm

import matplotlib.pyplot as plt

In [6]:
### load in the full acc data

os.chdir("C:/Users/meerkat/Documents/social_sleep/DATA/gps_acc_data/")

acc_dat = pd.read_csv('acc_dat.csv', header=0)

In [7]:
## only keep the necessary columns
acc_dat = acc_dat[ ['tag', 'corr_local_timestamp', 'acc_samp_freq', 'acc'] ]

In [8]:
# turn the timestamp into a datetime element
acc_dat[ 'corr_local_timestamp' ] = pd.to_datetime( acc_dat[ 'corr_local_timestamp' ] )

In [9]:
# save the path to the folder containing the night folders which contain .txt files with their respective processed videos
root = 'Z:/baboon/archive/processed/video/thermal/2019_summer/cliff_data/mp4_files/viewpoint_1/T1020'


In [10]:
# change the working directory to this folder
os.chdir( root )

In [11]:
# make a list of the night folders within this directory
nights = os.listdir()

In [12]:
# save the tag names
tag_names = SortedSet( acc_dat[ 'tag'] )

In [13]:
# this is the time in seconds of acc that you want to keep before a video starts and after a video ends
buffer = dt.timedelta( seconds = 120 ) 

In [14]:
# for each night
for night in nights:
    
    # save the path to this night's folder
    folder_path = os.path.join( root, night )
    
    # change the working directory to this night's folder
    os.chdir( folder_path )
    
    # make a list of the .txt files in the folder
    txt_files = glob.glob( "*.txt" )
    
    for txt_file in txt_files:
        
        hour_name = txt_file.split( '.' )[ 0 ]
        
        txt_filename = os.path.join( folder_path, txt_file )
        
        with open(txt_filename) as f:
            lines = f.readlines()
    
        frame_timestamps = list( map( lambda x: x.split( "'")[ 1 ].split( '.' ) [ 0 ].split( "_", 1 )[ 1 ] , lines ) )
        frame_timestamps = list( map( lambda x: dt.datetime.strptime(x, '%Y%m%d_%H%M%S%f'), frame_timestamps ) ) 

        frame_local_timestamps = list( map( lambda x: x + dt.timedelta( hours = 3 ), frame_timestamps ) )

        start_time = frame_local_timestamps[ 0 ]

        end_time = frame_local_timestamps[ ( len( frame_local_timestamps ) - 1 ) ]


        # these should be the same
        print( hour_name )
        print( start_time )

        # trim the acc data to the time of this video (plus the buffer before the start of the video and the buffer after the end of the video)
        sub_ved = acc_dat[ ( acc_dat[ 'corr_local_timestamp' ] > ( start_time - buffer ) ) & ( acc_dat[ 'corr_local_timestamp' ] < ( end_time + buffer ) ) ]
        
        # make a directory
        os.makedirs( 'C:/Users/meerkat/Documents/thermal_baboon/DATA/gps_acc_data/parsed_acc_for_animation/' + night + '/' + hour_name + '/', exist_ok = True )

        os.chdir( 'C:/Users/meerkat/Documents/thermal_baboon/DATA/gps_acc_data/parsed_acc_for_animation/' + night + '/' + hour_name + '/' )

        for tag in tag_names:

            print( tag )

            tag_dat = sub_ved[ sub_ved[ 'tag' ] == tag ]

            parsed_dat = pd.DataFrame( [] )

            for i in range( tag_dat.shape[ 0 ] ):

                if tag_dat.iloc[ i, 2 ] == 12:

                    to_add = pd.DataFrame( {'tag': tag, 'corr_local_timestamp': pd.date_range( tag_dat.iloc[ i, 1 ], tag_dat.iloc[ i, 1 ] + dt.timedelta( seconds = 1 ), 13 )[ :12 ], 'x': tag_dat[ 'acc' ].iloc[ i ].split( ' ' )[ 0 :: 3 ], 'y': tag_dat[ 'acc' ].iloc[ i ].split( ' ' )[ 1 :: 3 ], 'z': tag_dat[ 'acc' ].iloc[ i ].split( ' ' )[ 2 :: 3 ] } ) 

                    parsed_dat = pd.concat( [ parsed_dat, to_add ] )

                elif tag_dat.iloc[ i, 2 ] == 20:

                    to_add = pd.DataFrame( {'tag': tag, 'corr_local_timestamp': pd.date_range( tag_dat.iloc[ i, 1 ], tag_dat.iloc[ i, 1 ] + dt.timedelta( seconds = 2 ), 41 )[ :40 ], 'x': tag_dat[ 'acc' ].iloc[ i ].split( ' ' )[ 0 :: 3 ], 'y': tag_dat[ 'acc' ].iloc[ i ].split( ' ' )[ 1 :: 3 ], 'z': tag_dat[ 'acc' ].iloc[ i ].split( ' ' )[ 2 :: 3 ] } ) 

                    parsed_dat = pd.concat( [ parsed_dat, to_add ] )

                elif tag_dat.iloc[ i, 2 ] == 56.23:

                    to_add = pd.DataFrame( {'tag': tag, 'corr_local_timestamp': pd.date_range( tag_dat.iloc[ i, 1 ], tag_dat.iloc[ i, 1 ] + dt.timedelta( seconds = 1 ), 56.23 )[ :40 ], 'x': tag_dat[ 'acc' ].iloc[ i ].split( ' ' )[ 0 :: 3 ], 'y': tag_dat[ 'acc' ].iloc[ i ].split( ' ' )[ 1 :: 3 ], 'z': tag_dat[ 'acc' ].iloc[ i ].split( ' ' )[ 2 :: 3 ] } ) 

                    parsed_dat = pd.concat( [ parsed_dat, to_add ] )

            parsed_dat.to_csv( str( tag ) + '_' + hour_name + '.csv' )


20190731_150008934000
2019-07-31 18:00:08.934000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190731_160009036000
2019-07-31 19:00:09.036000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190731_170009174000
2019-07-31 20:00:09.174000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190731_180009337000
2019-07-31 21:00:09.337000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190731_190009467000
2019-07-31 22:00:09.467000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190731_200009580000
2019-07-31 23:00:09.580000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6

6914
6915
6921
6924
6927
6932
6933
6934
20190804_173009399000
2019-08-04 20:30:09.399000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190804_183009549000
2019-08-04 21:30:09.549000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190804_193009623000
2019-08-04 22:30:09.623000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190804_203009688000
2019-08-04 23:30:09.688000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190804_213009750000
2019-08-05 00:30:09.750000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190804_223009866000
2019-08-05 01:30:09.866000
2428
2433
2434
2

6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190810_143003016000
2019-08-10 17:30:03.016000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190811_133005954000
2019-08-11 16:30:05.954000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190811_143006006000
2019-08-11 17:30:06.006000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190811_153006074000
2019-08-11 18:30:06.074000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190811_163006140000
2019-08-11 19:30:06.140000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190811_173006203000
2019-08-11 20:30:0

2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190818_141102437000
2019-08-18 17:11:02.437000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190818_151102522000
2019-08-18 18:11:02.522000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190819_132001056000
2019-08-19 16:20:01.056000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190819_142001179000
2019-08-19 17:20:01.179000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
20190819_152001210000
2019-08-19 18:20:01.210000
2428
2433
2434
2436
2441
2447
2450
2451
2454
2455
6890
6891
6892
6894
6897
6898
6900
6910
6914
6915
6921
6924
6927
6932
6933
6934
